In [ ]:
import numpy as np
from keras.applications.densenet import DenseNet121
import keras.backend as K
import cv2
import os
from keras.callbacks import LearningRateScheduler,ReduceLROnPlateau,CSVLogger,ModelCheckpoint
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score,roc_auc_score, cohen_kappa_score
from keras.callbacks import Callback
from keras.layers import Dense
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [ ]:
#Custom callback function to calculate average ROC,F1 and Kappa score
class roc_callback(Callback):
    def __init__(self,validation_data):
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]


    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):

        if(epoch%1==0):
            print("Calc Roc")
            all_rocs = []
            y_pred_val = self.model.predict(self.x_val,verbose=1)

            try:
                roc_val = roc_auc_score(self.y_val, y_pred_val)
                all_rocs.append(roc_val)
            except:
                pass

            all_rocs = np.array(all_rocs)
            mean_roc = np.mean(all_rocs)
            
            Y_val_kappa=[np.argmax(i) for i in self.y_val]
            y_pred_kappa=[np.argmax(i) for i in y_pred_val]
                        
            avg_f1 = f1_score(Y_val_kappa, y_pred_kappa, average='weighted')
            kappa_score = cohen_kappa_score(Y_val_kappa, y_pred_kappa) 
            
            print("Mean ROC VAL {0}".format(mean_roc))
            print("AVG F1 {0}".format(avg_f1))
            print("KAPPA {0}".format(kappa_score))
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [ ]:
#X, Y - Load numpy arrays

In [ ]:
X_train,X_val,Y_train,Y_val=train_test_split(X,Y,test_size=0.2,random_state=42)

In [ ]:
y_train=np.array(Y_train)
#y_train[np.where(y_train==1)]=1
y_train[np.where(y_train==2)]=1

y_val=np.array(Y_val)
# y_val[np.where(y_val==1)]=0
y_val[np.where(y_val==2)]=1

In [ ]:
#Conversion of labels into One-hot encoding
Y_train=to_categorical(y_train,num_classes=2)
Y_val=to_categorical(y_val,num_classes=2)

In [ ]:
roc_call = roc_callback((X_val,Y_val))

In [ ]:
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, RandomBrightnessContrast, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, Flip, OneOf, Compose, RandomCrop, CenterCrop
)

aug = CLAHE(p=1)

X=[]
for i in X_train:
    image = aug(image=i)['image']
    X.append(image)
    
X=np.array(X)
print (X.shape)
np.save('./X_clahe_fold3.npy', X)

In [ ]:
X_train=np.concatenate([X_train, np.load('./X_horizontal_flip_fold3.npy', mmap_mode='r'), np.load('./X_clahe_fold3.npy', mmap_mode='r')])

In [ ]:
Y_train=np.concatenate([Y_train, Y_train, Y_train])

In [ ]:
Y_train.shape

In [ ]:
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, RandomBrightnessContrast, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, Flip, OneOf, Compose, RandomCrop, CenterCrop
)

aug = OneOf([
        HorizontalFlip(p=0.5),
        CLAHE(p=0.8)
]
)

def batch_generator(X,y, batch_size=32):
    '''
    Return a random image from X, y
    '''
    
    while True:
        # choose batch_size random images / labels from the data
        #idx = np.random.randint(0, X.shape[0], batch_size)
        i=0
        for idx in np.arange(32,X.shape[0], batch_size):
        
            im = np.array(X[i:idx])
            label = np.array(y[i:idx])
            for idx, image in enumerate(im):
                image2 = aug(image=image)['image']
                image2 = image2[0:130, 40:170]
                image2 = cv2.resize(image2, (224,224))
                im[idx] = image2
                



            yield im,label
            i = idx


In [ ]:
w=10
h=10
fig=plt.figure(figsize=(16,16))
columns = 4
rows = 5
for i in range(1, columns*rows +1):
    #img = np.random.randint(1,5000)
    fig.add_subplot(rows, columns, i)
    plt.imshow(X)
plt.show()

In [ ]:
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, RandomBrightnessContrast, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, Flip, OneOf, Compose, RandomCrop, CenterCrop
)


def augment_and_show(aug, image):
    image = aug(image=image)['image']
    return image
#     plt.figure(figsize=(10, 10))
#     plt.imshow(image)
    
aug = HorizontalFlip(p=1)
# augment_and_show(aug, X_train[0].astype('uint8'))

In [ ]:
w=10
h=10
fig=plt.figure(figsize=(16,16))
columns = 4
rows = 5
for i in range(1, columns*rows +1):
    #img = np.random.randint(1,5000)
    fig.add_subplot(rows, columns, i)
    a=augment_and_show(aug, X_train[i].astype('uint8'))
    plt.imshow(a)
plt.show()

In [ ]:
import albumentations
def augment_and_show(aug, image):
    image = aug(image=image)['image']
    return image
#     plt.figure(figsize=(10, 10))
#     plt.imshow(image)
    

aug = CLAHE(p=1)
# augment_and_show(aug, X_train[0].astype('uint8'))

In [ ]:
w=10
h=10
fig=plt.figure(figsize=(16,16))
columns = 4
rows = 5
for i in range(1, columns*rows +1):
    #img = np.random.randint(1,5000)
    fig.add_subplot(rows, columns, i)
    a=augment_and_show(aug, X_train[i].astype('uint8'))
    plt.imshow(cv2.resize(a, (224,224)))
plt.show()

In [ ]:
w=10
h=10
fig=plt.figure(figsize=(16,16))
columns = 4
rows = 5
for i in range(1, columns*rows +1):
    #img = np.random.randint(1,5000)
    fig.add_subplot(rows, columns, i)
#     a=augment_and_show(aug, X_train[i].astype('uint8'))
    a=X_train[i][0:130, 40:170].astype('uint8')
    plt.imshow(a)
#     plt.imshow(cv2.resize(a, (224,224)))
plt.show()

In [ ]:
from imgaug import augmenters as iaa

aug = iaa.Sequential([
    iaa.OneOf([ ## geometric transform
        iaa.Affine(
            scale={"x": (0.98, 1.02), "y": (0.98, 1.04)},
            translate_percent={"x": (-0.02, 0.02), "y": (-0.04, 0.04)},
            rotate=(-2, 2),
            shear=(-1, 1),
        ),
        iaa.PiecewiseAffine(scale=(0.001, 0.025)),
    ]),
    iaa.OneOf([ ## brightness or contrast
        iaa.Multiply((0.9, 1.1)),
        iaa.ContrastNormalization((0.9, 1.1)),
    ]),
    iaa.OneOf([ ## blur or sharpen
        iaa.GaussianBlur(sigma=(0.0, 0.1)),
        iaa.Sharpen(alpha=(0.0, 0.1)),
    ]),
])

def batch_generator(X,y, batch_size=32):
    while True:
        # choose batch_size random images / labels from the data
        #idx = np.random.randint(0, X.shape[0], batch_size)
        i=0
        for idx in np.arange(32,X.shape[0], batch_size):
        
            im = np.array(X[i:idx])
            label = np.array(y[i:idx])
            im = aug.augment_images(im)    #In-place flips
            
            yield im,label
            i = idx
        

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(
    horizontal_flip=True,
    #vertical_flip=True,
    rotation_range=5,
    #width_shift_range=0.2,
    #height_shift_range=0.2
)

train_datagen.fit(pad_X_train)

In [ ]:
#Model declaration:Densenet 
dense_model = DenseNet121(weights='imagenet',include_top=False,input_shape=(224,224,3),pooling='avg')
preds = Dense(3,activation='sigmoid')(dense_model.output)
# preds = Dense(5,activation='softmax')(dense_model.output)
model = Model(dense_model.input,preds)
# model.load_weights("categorical_3class_with_3class_wt_trained_on_fold23.hdf5")
model.layers.pop()
preds = Dense(2,activation='softmax')(model.layers[-1].output)
model2 = Model(dense_model.input, output=[preds])
model2.load_weights("complete_data_categorical.hdf5")


In [ ]:
adam=Adam(lr=1e-5)
model2.compile(optimizer=adam,loss='binary_crossentropy',metrics=['acc'])

In [ ]:
#Callback for saving model weights based on minimum Validation loss 
model_checkpoint = ModelCheckpoint(
        os.path.join('./', 'augmentation_test_fold3.hdf5'),
        monitor='val_acc', mode='max',save_best_only=True, verbose=1)

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.1,patience=6, min_delta=0.0001, verbose=1, min_lr=1e-8)

#Callback for storing logs of learning rate, loss and accuracy
csvlogger=CSVLogger('fold23.csv')

callbacks = [model_checkpoint, roc_call,reduce_lr,csvlogger]

In [ ]:
#Caluculation of class weights based on count of samples in training
from sklearn.utils import class_weight
y_true=np.argmax(Y_train,axis=-1)
weights = class_weight.compute_class_weight('balanced',np.unique(y_true),y_true)
print(weights)

In [ ]:
#Fitting model on Train and Validation dataset
model2.fit(X_train, Y_train, verbose=1, validation_data=(X_val, Y_val),epochs=300, callbacks=callbacks, shuffle=True, class_weight=weights)

In [ ]:
model2.load_weights('./augmentation_test_fold3.hdf5')

In [ ]:
X_val=np.load("dataset/x_test.npy")
Y_val=np.load("y_test.npy")

In [ ]:
p3=model2.predict(X_val, verbose=1)

In [ ]:
p2=p3

In [ ]:
p1=p3

In [ ]:
final=np.concatenate([p1,p2,p3])

In [ ]:
final

In [ ]:
y_pred = []
for i in p:
    if i[1]>0.025:
        y_pred.append(1)
    else:
        y_pred.append(0)
        
Y[np.where(Y==2)]=1

In [ ]:
print (classification_report(Y, y_pred, target_names=['Healthy', 'Pathology']))

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

confusion_matrix(Y, y_pred)

In [ ]:
pd.DataFrame(y_pred).to_csv('./predictions801.csv', index=False)

In [ ]:
preds= pd.read_csv('./predictions8001.csv')

In [ ]:
preds

In [ ]:
p1=y_pred       #80,58,0.0292

In [ ]:
p2=y_pred       

In [ ]:
roc_auc_score(to_categorical(y1), p3)

In [ ]:
p

In [ ]:
# probs = model.predict_proba(X_test)
import sklearn.metrics as metrics
preds = p3[:,1]
y_true = y1
fpr, tpr, threshold = metrics.roc_curve(y_true, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
# plt.savefig('3_class_ROC_curve_1e-6.png',dpi=200)
plt.show()

In [ ]:
confusion_matrix(y1, y_pred)

In [ ]:
p=np.concatenate([p1,p2,p3])

In [ ]:
y_pred = []
for i in p:
    if i[1]>0.0115:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [ ]:
pd.DataFrame(np.array(y_pred)).to_csv('./final_predictions.csv')